In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('project_data/remuneracao_quadros_tecnicos_raw.csv', sep=';')

In [3]:
df.sample(5)

,Unnamed: 0,REGISTRO,VINCULO,NOME,CARGO_BASICO,REF_CARGO_BAS,SEGMENTO,GRUPO,SUBGRUPO,ESCOL_CARGO_BASICO,...,dc_cargo_comissionado,vl_remuneracao_mensal,vl_outro_elemento_remuneracao,vl_remuneracao_total_bruta,cd_caso_especial,dc_tipo_logradouro_servidor,nm_logradouro_servidor,nr_logradouro_servidor,tx_complemento_logradouro_servidor,qt_hora_jornada_semanal
1043,1107,7266634,5,ROSIMEIRE LOBATO,ANALISTA DE MEIO AMBIENTE NIVEL II,Q6,MEIO AMBIENTE,QAA,SUPERIOR,SUPERIOR COMPLETO,...,COORDENADOR DE PROJETOS,"R$ 8.077,17","R$ 412,23","R$ 8.489,40",NaN,Rua,DO PARAISO,387.0,NaN,40 H
2209,2350,7902719,1,VANDA APARECIDA FERREIRA,ANALISTA ASSISTENCIA DESENVOLVIMENTO SOCIAL NI...,Q5,SERVICO SOCIAL,QAA,SUPERIOR,SUPERIOR COMPLETO,...,NaN,"R$ 7.215,11","R$ 353,34","R$ 7.568,45",NaN,Rua,BANDEIRANTES,55.0,NaN,30 H ESP ASST SOC
3367,3557,8593787,1,CARLOS AILTON DOS SANTOS JUNIOR,ANALISTA ASSISTENCIA DESENVOLVIMENTO SOCIAL NI...,Q1,SERVICO SOCIAL,QAA,SUPERIOR,SUPERIOR COMPLETO,...,COORDENADOR I,"R$ 6.753,31","R$ 1.058,85","R$ 7.812,16",NaN,Rua,SOLDADO JOSE ANTONIO MOREIRA,546.0,NaN,40 H
901,958,6878997,1,VANDO FRANCISCO DE JESUS,AUDITOR FISCAL TRIBUTARIO MUNICIPAL NII,QPAT-8,NaN,QPAT,SUPERIOR,SUPERIOR COMPLETO,...,NaN,"R$ 24.175,55","R$ 8.505,39","R$ 32.680,94",NaN,Rua,LIBERO BADARO,190.0,9Âº ANDAR,44 H
1282,1383,7536135,1,PATRICIA BONACIN SANCHEZ,"PROFISSIONAL ENG, ARQ, AGRONOMIA,GEOLOGIA NIVE...",QEAG7,ENGENHARIA,QEAG,SUPERIOR,SUPERIOR COMPLETO,...,NaN,"R$ 9.290,79","R$ 412,23","R$ 9.703,02",NaN,Rua,DO PARAISO,387.0,2Âº ANDAR,40 H


In [4]:
df.drop('Unnamed: 0', axis = 1, inplace=True)

O cargo base da planilha de remuneração está com problemas de encoding. Por isso, não dá para comparar com ==, mas o tamanho é o mesmo

In [5]:
len(df['dc_cargo_base'].unique()) == len(df['CARGO_BASICO'].unique())

True

No entanto, apesar de serem as mesmas categorias, caso as cruzemos percebemos que há alguns servidores no qual o nível mudou de uma base para outra (foram promovidos).

O código abaixo checa isso.

In [6]:
def cruzar_descricao_cargos(df, mostrar_apenas_mais_de_um = True):
    
    cross_tabs = pd.crosstab(df['dc_cargo_base'], df['CARGO_BASICO'])
    for col in cross_tabs:
        cross_tabs[col] = cross_tabs[col].apply(lambda x: True if x else False)
    if mostrar_apenas_mais_de_um:
        mais_de_um = cross_tabs[cross_tabs.apply(lambda row: row.sum()>1, axis=1).values]
        cols = []
        for col in mais_de_um:
            if mais_de_um[col].sum()>0:
                cols.append(col)

        mais_de_um = mais_de_um[cols]
        return mais_de_um.T
    return cross_tabs

In [7]:
cruzar_descricao_cargos(df)

dc_cargo_base,ANALISTA DE INFORMACOES CULTURA E DESPORTO NIVEL I,ANALISTA DE MEIO AMBIENTE NIVEL I
CARGO_BASICO,,
ANALISTA DE INFORMACOES CULTURA E DESPORTO NIVEL I,True,False
ANALISTA DE INFORMACOES CULTURA E DESPORTO NIVEL II,True,False
ANALISTA DE MEIO AMBIENTE NIVEL I,False,True
ANALISTA DE MEIO AMBIENTE NIVEL II,False,True


Precisamos arrumar isso, pois caso contrário a variável de nível pode ficar defasada e não performar corretamente no modelo.

In [8]:
for i, row in df.iterrows():
    
    if row['CARGO_BASICO']=='ANALISTA DE INFORMACOES CULTURA E DESPORTO NIVEL II' and \
    row['dc_cargo_base']=='ANALISTA DE INFORMACOES CULTURA E DESPORTO NIVEL I':
        df.loc[i, 'CARGO_BASICO'] = 'ANALISTA DE INFORMACOES CULTURA E DESPORTO NIVEL I'
        df.loc[i, 'nivel_limpo'] = 'I'

In [9]:
for i, row in df.iterrows():
    
    if row['CARGO_BASICO']=='ANALISTA DE MEIO AMBIENTE NIVEL II' and \
    row['dc_cargo_base']=='ANALISTA DE MEIO AMBIENTE NIVEL I':
        df.loc[i, 'CARGO_BASICO'] = 'ANALISTA DE MEIO AMBIENTE NIVEL I'
        df.loc[i, 'nivel_limpo'] = 'I'

In [10]:
df[(df['CARGO_BASICO']=='ANALISTA DE INFORMACOES CULTURA E DESPORTO NIVEL II')&
    (df['dc_cargo_base']=='ANALISTA DE INFORMACOES CULTURA E DESPORTO NIVEL I')][['CARGO_BASICO','dc_cargo_base', 'nivel_limpo']]

,CARGO_BASICO,dc_cargo_base,nivel_limpo


In [11]:
#a base de remuneração tem duas colunas iguais também
(df['dc_cargo']==df['dc_cargo_base']).all()

True

Como a divergência era apenas no "nível" do cargo, poderemos manter a coluna de cargo limpo que geramos anteriormente.

Com isso, podemos remover ambas as colunas de cargo, mantendo apenas o "cargo_limpo" e o "nivel"

In [12]:
df.drop(['dc_cargo_base', 'dc_cargo', 'CARGO_BASICO'], axis = 1, inplace=True)

Como podemos ver abaixo, há mais pessoas com cargo em comissão na base cadastral do que na base de remuneração. Isso pode acontecer pelas mesmas razões da mudança de nível acima: hoje a pessoa está com cargo em comissão, mas há um mês, quando fechou o salário dela, ela não tinha. Por isso o correto é manter o cargo em comissão da base de remuneração.

In [13]:
df['dc_cargo_comissionado'].isnull().sum()> df['CARGO_COMISSAO'].isnull().sum()

True

In [14]:
#XOR para quando o valor é nulo - ou seja quando um é nulo e o outro não
df[(df['CARGO_COMISSAO'].isnull()^df['dc_cargo_comissionado'].isnull())][['CARGO_COMISSAO', 'dc_cargo_comissionado']]

,CARGO_COMISSAO,dc_cargo_comissionado
224,COORDENADOR V,NaN
846,ASSESSOR TECNICO I,NaN
849,COORDENADOR,NaN
862,CONSELHEIRO JULGADOR,NaN
869,COORDENADOR,NaN
...,...,...
2922,ASSESSOR TECNICO I,NaN
2924,COORDENADOR,NaN
2925,ASSESSOR TECNICO I,NaN
2926,ASSESSOR TECNICO I,NaN


Temos também algumas colunas repetidas entre as bases, que vale a pena remover

In [15]:
df.drop('CARGO_COMISSAO', axis=1, inplace=True)

In [16]:
(df['REGISTRO']==df['cd_registro_funcional']).all()

True

In [17]:
df.drop('cd_registro_funcional', axis = 1, inplace=True)

In [18]:
(df['VINCULO']==df['cd_tipo_vinculo']).all()

True

In [19]:
df.drop('cd_tipo_vinculo', axis=1, inplace=True)

In [20]:
(df['NOME']==df['nm_servidor']).all()

False

No caso dos nomes, alguns estão mascarados na base de remuneração.

Vamos manter o nome da base cadastral.

In [21]:
df[df['NOME']!=df['nm_servidor']][['NOME', 'nm_servidor']]

,NOME,nm_servidor
17,JOSE MAURO RODRIGUES,SERVIDOR PUBLICO MUNICIPAL
216,CASSIO VIEIRA PEREIRA DOS SANTOS,REGISTRO FUNCIONAL=5516943
851,PATRICIA GANDRACHAO BENOTTI,REGISTRO FUNCIONAL=6869645
872,CLARICE KEIKO AOKI,REGISTRO FUNCIONAL=6873642
901,VANDO FRANCISCO DE JESUS,REGISTRO FUNCIONAL=6878997
908,SHEILA CRISTINA TAMBARA,REGISTRO FUNCIONAL=6883036
955,LUCIANA SALZANI,REGISTRO FUNCIONAL=6935192
1027,ANTONIO BASTOS TORRES LIMA,SERVIDOR PUBLICO MUNICIPAL
1199,LUIZ CARLOS PIRES,SERVIDOR PUBLICO MUNICIPAL
1201,JOSE FERNANDO PIRES DOS SANTOS,SERVIDOR PUBLICO MUNICIPAL


In [22]:
df.drop('nm_servidor', axis = 1, inplace=True)

No caso das secretarias, temos dois problemas: 
* a pessoa pode ter sido transferida após o fechamento da folha de pagamento;
* algumas secretarias a Unidade Orçamentária possui outro nome: por exemplo a Secretaria de Governo está vinculada ao Gabinete do Prefeito

In [23]:
(df['SECRET_SUBPREF']==df['dc_secretaria']).all()

False

In [24]:
#apenas a secretaria de governo e a casa civil não estão na base de remuneração
[item for item in df['SECRET_SUBPREF'].unique() if item not in df['dc_secretaria'].unique()]

['SECRETARIA DE GOVERNO MUNICIPAL', 'CASA CIVIL', nan]

In [25]:
len(df['SECRET_SUBPREF'].unique()) - len(df['dc_secretaria'].unique())

3

In [26]:
df['dc_secretaria'].isnull().sum()

0

In [27]:
df['SECRET_SUBPREF'].isnull().sum() #há um nulo aqui também

1

In [28]:
diferentes = df[df['SECRET_SUBPREF']!=df['dc_secretaria']][['SECRET_SUBPREF', 'dc_secretaria']]
diferentes[~diferentes.duplicated()]

,SECRET_SUBPREF,dc_secretaria
18,SECRETARIA DE GOVERNO MUNICIPAL,GABINETE DO PREFEITO
330,CASA CIVIL,GABINETE DO PREFEITO
2656,SUBPREFEITURA IPIRANGA,SUBPREFEITURA LAPA
2807,NaN,SECRETARIA MUNICIPAL DE ASSIST E DESENV SOCIAL


Por isso, vamos manter a secretaria da base de remuneração

In [29]:
df.drop('SECRET_SUBPREF', axis =1, inplace=True)

Escolher essa coluna também é positivo, pois podemos manter o endereço da base de remuneração, que bate com a descrição da secretaria dessa base mesma base (caso usássemos o nome da secretaria de cadastro poderia haver divergências, como no caso do servidor que mudou da subprefeitura da Lapa para a Ipiranga).

In [30]:
df['tp_servidor'].unique()

array(['ATIVOS'], dtype=object)

Todos os servidores são do quadro de servidores ativos, então podemos remover essa coluna.

In [31]:
df.drop('tp_servidor', axis =1, inplace=True)

In [32]:
df['SUBGRUPO'].unique()

array(['SUPERIOR'], dtype=object)

O mesmo vale para o subgrupo do cargo base, pois se trata apenas de servidores de nível superior, como visto no outro notebook. 

In [33]:
df.drop('SUBGRUPO',axis=1, inplace=True)

In [34]:
df['ESCOL_CARGO_BASICO'].unique()

array(['SUPERIOR COMPLETO'], dtype=object)

E, logicamente, também para a escolaridade do cargo básico.

In [35]:
df.drop('ESCOL_CARGO_BASICO', axis=1, inplace=True)

In [36]:
df['cd_caso_especial'].unique()

array([nan,  5.])

Temos apenas um servidor marcado como "caso especial", que entrou na Prefeitura em 1986 e que hoje ganha acima do teto.

Como a própria base o registra como um caso especial, achamos melhor retirá-lo da base para não enviesar o modelo.

In [37]:
df[df['cd_caso_especial']==5]

,REGISTRO,VINCULO,NOME,REF_CARGO_BAS,SEGMENTO,GRUPO,REF_CARGO_COM,ESCOL_CARGO_COMISSAO,DATA_INICIO_EXERC,REL_JUR_ADM,...,dc_cargo_comissionado,vl_remuneracao_mensal,vl_outro_elemento_remuneracao,vl_remuneracao_total_bruta,cd_caso_especial,dc_tipo_logradouro_servidor,nm_logradouro_servidor,nr_logradouro_servidor,tx_complemento_logradouro_servidor,qt_hora_jornada_semanal
284,5681138,1,GERALDO REA RABELLO SAMPAIO,QPAT-9,NaN,QPAT,NaN,NaN,17/01/1986,EFETIVO,...,NaN,"R$ 28.285,10","R$ 14.083,92","R$ 42.369,02",5.0,Rua,LIBERO BADARO,190.0,8Âº ANDAR,44 H


In [38]:
caso_especial = df[df['cd_caso_especial']==5].index

In [39]:
df.drop(caso_especial, inplace=True)

In [ ]:
df['cd_caso_especial'].unique()

In [ ]:
df.drop('cd_caso_especial', axis=1, inplace=True)

In [ ]:
df.keys()

In [ ]:
from collections import OrderedDict

In [ ]:
renomear = OrderedDict(
    REGISTRO = 'registro_funcional',
    VINCULO = 'num_vinculo', #pessoa pode ter sido contratado pela prefeitura mais de uma vez
    NOME = 'nome_servidor',
    cargo_limpo = 'dc_cargo_base',
    GRUPO = 'grupo_cargo_base',
    nivel_limpo = 'nivel_cargo_base',
    REF_CARGO_BAS = 'referencia_cargo_base',
    SEGMENTO = 'segmento_cargo_base',
    dc_cargo_comissionado = 'dc_cargo_comissao',
    REF_CARGO_COM = 'referencia_cargo_comissao',
    ESCOL_CARGO_COMISSAO = 'escol_cargo_comissao',
    REL_JUR_ADM = 'relacao_juridica',
    qt_hora_jornada_semanal = 'qt_hora_jornada_semanal',
    dc_secretaria = 'dc_orgao',
    SETOR = 'dc_setor',
    ORGAO_EXT = 'dc_orgao_externo',
    qtd_servidores_orgao = 'qtd_servidores_orgao',
    dc_tipo_logradouro_servidor = 'dc_tipo_logradouro_orgao',
    nm_logradouro_servidor = 'nm_logradouro_orgao',
    nr_logradouro_servidor = 'nr_logradouro_orgao',
    tx_complemento_logradouro_servidor = 'tx_complemento_logradouro_orgao',
    DATA_INICIO_EXERC = 'dt_inicio_exercicio',
    ANO_NASCIMENTO = 'ano_nascimento',
    SEXO = 'sexo',
    RACA = 'raca',
    DEFICIENTE = 'pessoa_com_deficiencia',
    vl_remuneracao_mensal = 'vl_remuneracao_base_mensal',
    vl_outro_elemento_remuneracao = 'vl_outro_elemento_remuneracao',
    vl_remuneracao_total_bruta = 'vl_remuneracao_total_bruta',
)

In [ ]:
df.rename(renomear, axis = 1, inplace=True)

In [ ]:
df = df[renomear.values()].copy()

In [ ]:
df.to_csv('project_data/remuneracao_quadros_tecnicos.csv', sep=';', encoding='UTF-*')